# Introduction to Scipy: Interpolation and Integration

In this lecture, we will look at three other common sub-packages of Scipy:
[scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html),
[scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html),
and [scipy.linalg](https://docs.scipy.org/doc/scipy/reference/linalg.html).

## [SKIP] Interpolation

The simplest interpolation routine in [scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html) is [interp1d](http://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d):

In [1]:
from scipy.interpolate import interp1d

If we create a fake dataset:

In [2]:
import numpy as np
x = np.array([0., 1., 3., 4.])
y = np.array([0., 4., 3., 2.])

we can interpolate linearly by first creating an interpolating function:

In [3]:
f = interp1d(x, y)

and we can then interpolate to any value of x within the original bounds:

In [4]:
f(0.5)

array(2.)

In [5]:
f(3.3)

array(2.7)

It is also possible to interpolate to several values at the same time:

In [6]:
f(np.array([0.5, 1.5, 2.5, 3.5]))

array([2.  , 3.75, 3.25, 2.5 ])

If the interpolating function is called outside the original range, an error is raised:

In [7]:
#f(-1.)

You can change this behavior by telling ``interp1d`` to not give an error in this case, but to use a set value:

In [8]:
f = interp1d(x, y, bounds_error=False, fill_value=-10.)

In [9]:
f(-1.0)

array(-10.)

In [10]:
f(np.array([-1., 1., 3., 6.]))

array([-10.,   4.,   3., -10.])

By default, ``interp1d`` uses linear interpolation, but it is also possible to use e.g. cubic interpolation: 

In [11]:
f = interp1d(x, y, kind='cubic')
f(0.5)

array(2.583333)

For more information, see the documentation for [interp1d](http://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d). There are also other interpolation functions available (for example for spline interpolation), which you can read up about at [scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html).

## Integration

The available integration functions are listed at [scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html#module-scipy.integrate). You will notice there are two kinds of functions - those that integrate actual Python functions, and those that integrate numerical functions defined by Numpy arrays.

First, we can take a look at one of the functions that can integrate actual Python functions. If we define a function:

In [12]:
def simple_function(x):
    return 3. * x**2 + 2. * x + 1.

we can integrate it between limits using:

In [13]:
from scipy.integrate import quad
print(quad(simple_function, 1., 2.))

(10.999999999999998, 1.221245327087672e-13)


As described in the documentation for [quad](http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad), the first value returned is the integral, and the second is the error on the integral. If we had solved the integral analytically, we would expect 11, so the result is correct.

We can also define function values  as Numpy arrays:

In [14]:
x = np.linspace(1., 2., 1000)
y = simple_function(x)

And in this case we can use for example the [simps](http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simps.html#scipy.integrate.simps) function to integrate using Simpson's rule:

In [15]:
from scipy.integrate import simps
print(simps(y, x=x))

11.000000000501505


This can be very useful in cases where one wants to integral actual data that cannot be represented as a simple function or when the function is only available numerically.

Note that there is an issue on the [scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html#module-scipy.integrate) page - there should also be a menton of the ``trapz`` function which works similarly to ``simps`` but does trapezium integration:

In [16]:
from scipy.integrate import trapz
print(trapz(y, x=x))

11.000000501001502


## [SKIP] Exercise

Write a function that takes ``x``, and the parameters for a Gaussian (amplitude, displacement, width) and returns the value of the Gaussian at ``x``:

In [17]:

# your solution here


Use ``quad`` to compute the integral and compare to what you would expect.

In [18]:

# your solution here


Now create two arrays ``x`` and ``y`` that contain the Gaussian for fixed values ``x``, and try and compute the integral using ``simps``.

In [19]:

# your solution here


Compare this to what you found with ``quad`` and analytically.

## Linear Algebra

scipy is able to perform various linear algebra problems such as
[matrix equation solving](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html#scipy.linalg.solve),
[matrix norm computation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.norm.html#scipy.linalg.norm),
[kronecker product formation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.kron.html#scipy.linalg.kron),
as well as various decompositions, in particular [symmetric/Hermitian](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html#scipy.linalg.eigh),
and [unsymmetric](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html#scipy.linalg.eig) eigenvalue decomposition.

Here, we will have a closer look at eigenvalue decomposition. First, we create a symmetric random matrix $H$ of size $N$:


In [20]:
N = 5 # matrix size
H = np.random.rand(N,N) # unsymmetric matrix
H = H + H.T.conj() # symmetrize
print("H=",H)

H= [[0.527051 0.635361 1.865956 1.25765  1.103204]
 [0.635361 0.925908 0.33925  1.209904 1.489916]
 [1.865956 0.33925  0.880353 0.405535 0.667415]
 [1.25765  1.209904 0.405535 0.721885 1.257838]
 [1.103204 1.489916 0.667415 1.257838 0.589226]]


Let us check explicitly whether $H$ really is symmetric by printing out $\| H-H^\dagger\|_2$:


In [21]:
import scipy.linalg as la  # conventional abbreviation

print(f"||H-H'||={la.norm(H-H.T.conj()):.16f}", np.allclose(H,H.T.conj()))

||H-H'||=0.0000000000000000 True


Now, let's do the eigenvalue decomposition: $\mathbf H = \mathbf{U} \text{diag}(\mathbf E) \mathbf{U}^\dagger$


In [22]:
E,U = la.eigh(H)
print("the eigenvalues are ",E)
print("the eigenvectors are\n",U)

the eigenvalues are  [-1.428717 -0.819917 -0.29338   1.33353   4.852907]
the eigenvectors are
 [[ 0.754547 -0.00874  -0.187111 -0.402041  0.483668]
 [ 0.119213 -0.51464   0.526529  0.508088  0.430754]
 [-0.535325 -0.157028  0.295966 -0.670401  0.389533]
 [-0.343359 -0.213015 -0.750745  0.257026  0.455026]
 [-0.109449  0.815498  0.191162  0.254381  0.470884]]


let's print it in a nicer way



In [23]:
for i in range(N):
    print(f"i = {i}, eigenvalue={E[i]: .12f} and eigenvector=",U[:,i])

i = 0, eigenvalue=-1.428717333552 and eigenvector= [ 0.754547  0.119213 -0.535325 -0.343359 -0.109449]
i = 1, eigenvalue=-0.819917232400 and eigenvector= [-0.00874  -0.51464  -0.157028 -0.213015  0.815498]
i = 2, eigenvalue=-0.293380041521 and eigenvector= [-0.187111  0.526529  0.295966 -0.750745  0.191162]
i = 3, eigenvalue= 1.333530248842 and eigenvector= [-0.402041  0.508088 -0.670401  0.257026  0.254381]
i = 4, eigenvalue= 4.852906935858 and eigenvector= [0.483668 0.430754 0.389533 0.455026 0.470884]


let's check that the eigendecomposition is correct:



In [24]:
Hrecovered = U @ np.diag(E) @ U.T.conj()
print("|H - UEU'|= ",la.norm(Hrecovered-H))

|H - UEU'|=  1.0408451877571193e-14
